# ¿Existe sesgo de medio para presentar las noticias respecto de los mapuches? (pluralismo)

In [2]:
import numpy as np
import pandas as pd
#Cargar el dataset de tweets
df_mapuches = pd.read_csv('datasets/sophia_mapuche_v2.csv',delimiter="|", header=None)
df_medios = pd.read_csv('datasets/sophia_medios-chile-2017.csv',delimiter="|", header=None)

#selección de los mensajes
docs = df_mapuches[3].as_matrix()
len(docs)

1493

In [3]:
print(df_mapuches[1].value_counts())

 Mapuexpress         184
 biobio              125
 araucanianews        82
 eldesconcierto       69
 soytemuco            59
 AustralTemuco        58
 El_Ciudadano         49
 adnradiochile        41
 TVU_television       36
 laopinon             35
 rsumen               32
 CNNChile             27
 el_dinamo            26
 Temucodiario         26
 24HorasTVN           24
 RedMiVoz             24
 GAMBA_CL             23
 latercera            22
 thecliniccl          22
 uchileradio          19
 elmostrador          18
 nacioncl             16
 mercuriovalpo        15
 T13                  13
 prensaopal           13
 FortinOficial        12
 CHVNoticiascl        12
 ELCLARINDECHILE      11
 rtierrabella         11
 corrupcionchile      10
                    ... 
 pinguinodiario        1
 carta_abierta         1
 sancarlosonline       1
 eltipografo           1
 elsoldeiquique        1
 elobservatodo         1
 vlnradio              1
 HDtuiter              1
 ElPeriodista          1


In [4]:
import matplotlib
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
n_samples = 0.7
n_features = 1000
n_top_words = 20
dataset = docs
data_samples = dataset[:int(len(dataset)*n_samples)]
test_samples = dataset[int(len(dataset)*n_samples):]


stopWords = set(nltk.corpus.stopwords.words('spanish'))

print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=7,
                                max_features=n_features,
                                stop_words=stopWords)
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
t0 = time()
tf_test = tf_vectorizer.transform(test_samples)
print("done in %0.3fs." % (time() - t0))

perplexity = []

for i in range(1,12):
    n_topics = i

    #print("Fitting LDA models with tf features, "
    #      "n_samples=%d, n_features=%d n_topics=%d "
    #      % (n_samples, n_features, n_topics))

    lda = LatentDirichletAllocation(n_components=n_topics, max_iter=5,
                                    learning_method='online',
                                    learning_offset=50.,
                                    random_state=0)
    t0 = time()
    lda.fit(tf)

    train_gamma = lda.transform(tf)
    train_perplexity = lda.perplexity(tf, train_gamma)
    
    test_gamma = lda.transform(tf_test)
    test_perplexity = lda.perplexity(tf_test, test_gamma)
    
    perplexity.append(test_perplexity)
    
    #print('sklearn preplexity: train=%.3f, test=%.3f' %
    #      (train_perplexity, test_perplexity))

    #print("done in %0.3fs." % (time() - t0))
    #
plt.plot(range(1,12),perplexity,'-o')
plt.xlabel('Numero de clusters')
plt.ylabel('Perplexity')
plt.xticks(range(1,12))
plt.show()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/roberto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/roberto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Extracting tf features for LDA...
done in 0.405s.
Extracting tf features for LDA...
done in 0.150s.


/Users/roberto/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:812: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if doc_topic_distr != 'deprecated':
/Users/roberto/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:816: DeprecationWarning: Argument 'doc_topic_distr' is deprecated and is being ignored as of 0.19. Support for this argument will be removed in 0.21.
  DeprecationWarning)
/Users/roberto/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:816: DeprecationWarning: Argument 'doc_topic_distr' is deprecated and is being ignored as of 0.19. Support for this argument will be removed in 0.21.
  DeprecationWarning)
/Users/roberto/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:816: DeprecationWarning: Argument 'doc_topic_distr' is deprecated and is being ignored as of 0.19. Support for this argument will b

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import adjusted_rand_score

stemmer = SnowballStemmer("spanish")

# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('spanish')

# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [ ]:
#Aplicando Modelos Probabilistas de Tópicos y LDA
from sklearn.decomposition import LatentDirichletAllocation
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()
    
stopWords = set(nltk.corpus.stopwords.words('spanish'))


tf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=7,
                                stop_words=stopWords,tokenizer=tokenize_only, ngram_range=(1,1))
tf = tf_vectorizer.fit_transform(docs)
diccionario= tf_vectorizer.get_feature_names()


#Estimación de LDA con Bayes Variacional
lda = LatentDirichletAllocation(n_components=4, max_iter=10,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(tf)
#Cálculo de índice de ajuste de los datos
print(lda.perplexity(tf))
print("\nTopics in LDA model:")
print_top_words(lda, diccionario, 30)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

vectorizer = TfidfVectorizer(stop_words=stopWords,tokenizer=tokenize_and_stem,ngram_range=(1,1))
X = vectorizer.fit_transform(docs)
num_k = range(1, 12)
inertias = []

for k in num_k:
    # Create a KMeans instance with k clusters: model
    model = KMeans(n_clusters=k, n_init=10)

    # Fit model to samples
    model.fit(X)

    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)



# Plot ks vs inertias
plt.plot(num_k, inertias, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(num_k)
plt.show()

In [ ]:
#Utilizando Kmeans y tf-idf para realizar un primer agrupamiento de los mensajes
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

vectorizer = TfidfVectorizer(stop_words=stopWords,tokenizer=tokenize_and_stem,ngram_range=(1,1))
X = vectorizer.fit_transform(docs)

from matplotlib.pylab import hist, show
true_k = 4
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=10)
model.fit(X)
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    cluster_words = np.array(terms)[order_centroids[i, :15]]
    print('Cluster {}: {}'.format(i, ' '.join(cluster_words)))

#Generando algunos gráficos de los resultados
labels = model.predict(X)
plt.scatter(range(1493),labels,c=labels)
show()
hist(labels,bins=np.arange(-0.5,5.6,1),alpha=0.5,rwidth=0.9)
show()

In [1]:
import spacy
nlp = spacy.load('es')

def tokenize_lemmatize_filtering(text):
    filtered_lemmas = []
    doc = nlp(text)
    for token in doc:
        if (token.pos_ == 'NOUN' or token.pos_ == 'ADJ' or token.pos_ == "VERB"):
            print(token.pos_)
            filtered_lemmas.append(token.lemma_.lower())
            
        for w in doc.ents:
            print("Named Entities:",w,"--->",w.label)
            
        return filtered_lemmas
    
results = tokenize_lemmatize_filtering("El pequeño gato se encuantrea en la pieza.")
print(results)
    
    

[]
